# Data Exploration - Team Gambinos

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
complaints = pd.read_csv('../data/complaints.csv')

We decided as a team to adopt the following naming convention for consistency:

In [3]:
complaints.columns = [x.replace('Consumer ', '').lower().replace(' ', '_') for x in complaints.columns]
complaints['complaint_length'] = complaints['complaint_narrative'].apply(len)


In [4]:
complaints['issue'].value_counts()

Incorrect information on your report    229305
Attempts to collect debt not owed        73163
Communication tactics                    21243
Struggling to pay mortgage               17374
Fraud or scam                            12347
Name: issue, dtype: int64

In [5]:
complaints['complaint_narrative']

0         My name is XXXX XXXX this complaint is not mad...
1         I searched on XXXX for XXXXXXXX XXXX  and was ...
2         I have a particular account that is stating th...
3         I have not supplied proof under the doctrine o...
4         Hello i'm writing regarding account on my cred...
                                ...                        
353427           Collections account I have no knowledge of
353428    Dear CFPB Team, The reason for my complaint is...
353429    FRCA violations : Failing to Follow Debt Dispu...
353430    My Father, a XXXX XXXX  acquired an HECM rever...
353431    I have tried to contact cash app about a fraud...
Name: complaint_narrative, Length: 353432, dtype: object

In [6]:
seed = 123
for complaint in complaints.loc[complaints['issue'] == 'Incorrect information on your report', 'complaint_narrative'].sample(3, random_state=seed):
    print(complaint)
    print('-----------------------------')

I just reviewed a copy of my Experian credit report and found the following information to be an error. I am a victim of identity theft and did not make these charges. I reported the theft of my identity to the Federal Trade Commission.
-----------------------------
after my legal separation from my husband he started to open credit in my name with no permission I have a legal case against him but can not find him he has ruined my life his name is XXXX XXXX XXXX   remove I have disputed by mail with all three credit bureaus. act # XXXX owing an alleged {$2200.00} XXXX   XXXX  orig creditor XXXX
-----------------------------
Equifax report a collections account " XXXX XXXX XXXX ''. I have no knowledge of this account. I have not been furnished any proof or verifications of this account. I don't have any signed contract agreements with XXXX XXXX XXXX, verbal agreements nor any paperwork associated with them. A proper investigation according to FCRA was not conducted, word of mouth from t

In [7]:
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace(r'[xX][xX]+','', regex=True) #\s

In [18]:
complaints.groupby('issue')['complaint_narrative'].apply(lambda x: x.str.extractall(r'(\w+X+\w+)').groupby(0).size().reset_index(name='count'))


0  count
issue                                            
Attempts to collect debt not owed 0     X       8
                                  1    2X       6
                                  2    3X       4
                                  3    4X       1
                                  4    EX      19
...                                    ...    ...
Struggling to pay mortgage        36  r X       2
                                  37  s X       1
                                  38  t X       3
                                  39  y X       6
                                  40  } X       5

[251 rows x 2 columns]

In [9]:
complaints.loc[complaints['complaint_narrative'].str.find('TOXIC') != -1, 'complaint_narrative'].iloc[3]

'  and      , CA  NATIONSTAR/.   #  We need immediate assistance with our   property loan.     is the investor of the property and   is the servicer for the loan. \n\nOur property has a domestic water well and the water is unsafe to drink. The significant and very serious water contaminations in the aquifers that serve our water well include : Lead Hexavalent Chromium ( documented in the movie entitled   ) and Arsenic. These chemicals are listed on the CA Proposition 65 List of Contaminants that Cause Cancer or Reproductive Harm. Serial testing of the water well by a CA Certified Laboratory defined the contamination to be steady-state as we were informed by the US EPA. There is no filtration system that can reduce these high levels down to potable safe drinking water levels. Our home is now defined* as : HAZARDOUS TOXIC CONTAMINATED UNINHABITABLE POLLUTED NON-POTABLE WATER BLIGHTED AND A NUISANCE ( *as defined by the CA List of Contaminants Proposition 65 State of CA, EPA Office of Env

In [10]:
X = complaints[['complaint_narrative']]
y = complaints['issue']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [11]:
vect = CountVectorizer() # this is a transformer

X_train_vec = vect.fit_transform(X_train['complaint_narrative'])
X_test_vec = vect.transform(X_test['complaint_narrative'])

In [12]:
vect.vocabulary_

{'filed': 27146,
 'an': 7429,
 'identity': 32631,
 'theft': 61712,
 'report': 52668,
 'when': 68070,
 'learned': 37604,
 'that': 61489,
 'someone': 57873,
 'was': 67575,
 'using': 66093,
 'my': 41860,
 'during': 23183,
 'incarceration': 33621,
 'submitted': 59636,
 'documentation': 22499,
 'about': 4660,
 'and': 7488,
 'transunion': 63495,
 'disregarded': 22092,
 'it': 36087,
 'they': 62022,
 'didn': 21149,
 'even': 25307,
 'block': 11710,
 'the': 61560,
 'account': 5017,
 'for': 27883,
 '30': 2280,
 'days': 19304,
 'upon': 65849,
 'receiving': 50897,
 'information': 34327,
 'also': 7058,
 'received': 50868,
 'letter': 37891,
 'from': 28790,
 'irs': 35939,
 'stating': 58860,
 'tax': 60925,
 'return': 53813,
 'in': 33421,
 'name': 42068,
 'us': 65944,
 'dept': 20523,
 'of': 44068,
 'educatio': 23553,
 'acct': 5153,
 'on': 44416,
 '2021': 1750,
 'closed': 15146,
 'send': 56000,
 'written': 68946,
 'notification': 43323,
 'will': 68343,
 'be': 10726,
 'reported': 52683,
 'to': 62733,
 'co

In [13]:
len(vect.vocabulary_)

69470

In [14]:
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': np.array(X_train_vec.sum(axis = 0)).flatten()
})

word_counts.head()

,words,frequency
0,00,240345
1,000,1069
2,0000,10
3,00000,7
4,0000000000000000000000000000000000000000000000...,4


In [15]:
word_counts.sort_values('frequency', ascending=False).head(25)

,words,frequency
61560,the,1834415
62733,to,1491935
7488,and,1230803
41860,my,964352
44068,of,892744
61489,that,674232
62145,this,587835
18385,credit,552517
33421,in,533472
35946,is,529514
